# Machine learning algorithm to forecast intracompartmental pressure

## Train TensorFlow Lite regression algorithm

Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Train regression model

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load dataset fsr_data.csv
df = pd.read_csv('/content/drive/MyDrive/SEF 2023-2024/TinyML/training_two.csv')

# Prepare data for training
scaler = MinMaxScaler()
df[['fsr_1','fsr_2','fsr_3','fsr_4','fsr_5', 'ground_truth']] = scaler.fit_transform(df[['fsr_1','fsr_2','fsr_3','fsr_4','fsr_5', 'ground_truth']])

# Split data into features (X) and target (y)
X = df[['fsr_1','fsr_2','fsr_3','fsr_4','fsr_5']]
y = df['ground_truth']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define # of inputs
input_layer = tf.keras.Input(shape=(5))

# Create the model architecture
x = tf.keras.layers.Dense(64, activation='relu')(input_layer) # 64 neurons + relu for nonlinearity
x = tf.keras.layers.Dropout(0.5)(x) # prevent overfitting
x = tf.keras.layers.Dense(32, activation='relu')(x)  # 32 neurons + relu for nonlinearity
x = tf.keras.layers.Dropout(0.5)(x) # prevent overfitting

# Output layer
output_layer = tf.keras.layers.Dense(1)(x)

# Create the model
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('/content/drive/MyDrive/SEF 2023-2024/TinyML/model_two.tflite', 'wb') as f:
    f.write(tflite_model)

Epoch 1/10
5/5 [==============================] - 1s 60ms/step - loss: 0.2341 - val_loss: 0.1476
Epoch 2/10
5/5 [==============================] - 0s 15ms/step - loss: 0.2079 - val_loss: 0.0988
Epoch 3/10
5/5 [==============================] - 0s 11ms/step - loss: 0.2061 - val_loss: 0.0843
Epoch 4/10
5/5 [==============================] - 0s 10ms/step - loss: 0.1777 - val_loss: 0.0800
Epoch 5/10
5/5 [==============================] - 0s 15ms/step - loss: 0.1850 - val_loss: 0.0764
Epoch 6/10
5/5 [==============================] - 0s 10ms/step - loss: 0.1909 - val_loss: 0.0770
Epoch 7/10
5/5 [==============================] - 0s 15ms/step - loss: 0.1652 - val_loss: 0.0778
Epoch 8/10
5/5 [==============================] - 0s 11ms/step - loss: 0.1792 - val_loss: 0.0789
Epoch 9/10
5/5 [==============================] - 0s 12ms/step - loss: 0.1485 - val_loss: 0.0820
Epoch 10/10
5/5 [==============================] - 0s 11ms/step - loss: 0.1677 - val_loss: 0.0847



##Generate machine learning png architecture diagram


In [ ]:
tf.keras.utils.plot_model(model, "model.png")

In [ ]:
tf.keras.utils.plot_model(model, "model_with_shape.png", show_shapes=True)

## Save the TFLite model in .h format (to work with Arduino Sketch)

In [ ]:
tflite_model_path = '/content/drive/My Drive/SEF 2023-2024/TinyML/model_one.h'
with open(tflite_model_path, 'w') as f:  # Open the file in text mode
    f.write('#ifndef MODEL_H\n')
    f.write('#define MODEL_H\n\n')
    f.write('#include <cstdint>\n\n')
    f.write('const uint8_t model[] = {\n')
    for byte in tflite_model:
        f.write(hex(byte) + ', ')
    f.write('\n};\n\n#endif\n')

# Print the path for reference
print(f'TFLite model saved as {tflite_model_path}')

## Evaluate model performance (regression)

In [ ]:
# Model regression statistics

# Evaluate the Model
loss = model.evaluate(X_test, y_test)
print(f'Mean Squared Error on Test Set: {loss}')

# Predictions on Test Set
y_pred = model.predict(X_test)

# Calculate additional performance metrics
from sklearn.metrics import mean_absolute_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Absolute Error on Test Set: {mae}')
print(f'R-squared on Test Set: {r2}')

2/2 [==============================] - 0s 11ms/step - loss: 0.0966
Mean Squared Error on Test Set: 0.09663528203964233
2/2 [==============================] - 0s 5ms/step
Mean Absolute Error on Test Set: 0.29050559857098324
R-squared on Test Set: 0.0208584068893769


## Manually test trained model

In [ ]:
import numpy as np
import tensorflow as tf

# Load the TensorFlow Lite model
model_path = '/content/drive/My Drive/SEF 2023-2024/TinyML/model_two.tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Define a function to get input from the user
def get_user_input():
    # Ask the user for input values
    fsr_1 = float(input("Enter FSR 1 value: "))
    fsr_2 = float(input("Enter FSR 1 value: "))
    fsr_3 = float(input("Enter FSR 1 value: "))
    fsr_4 = float(input("Enter FSR 1 value: "))
    fsr_5 = float(input("Enter FSR 1 value: "))

    # Create a numpy array with the user input
    user_input = np.array([fsr_1,fsr_2,fsr_3,fsr_4,fsr_5,], dtype=np.float32).reshape(5, 1)
    return user_input

# Get user input
processed_data = get_user_input()

# Set input tensor
input_details = interpreter.get_input_details()
interpreter.set_tensor(input_details[0]['index'], processed_data)

# Run inference
interpreter.invoke()

# Get output tensor
output_details = interpreter.get_output_details()
output_data = interpreter.get_tensor(output_details[0]['index'])

# Process the output data as needed
print("Processed Input Data:", processed_data)
print("Output Data:", output_data)

Enter FSR 1 value: 80
Enter FSR 1 value: 80
Enter FSR 1 value: 80
Enter FSR 1 value: 80
Enter FSR 1 value: 80


ValueError: ignored